In [1]:
import numpy as np
import xarray as xr
import datetime as dt
import os
import matplotlib.pyplot as plt

In [2]:
# Years used for climatology
clim_years = [1991, 2020]

# Input/output directory
basepath='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW'
dir_forc = basepath+'/concatenated'#/NMME_SST/by_lead
# dir_forc = '/Volumes/Data_2TB/NMME/SST/by_lead/concatenated';
dir_clim = basepath+'/climatology'
# dir_clim = '/Volumes/Data_2TB/NMME/SST/by_lead/climatology';
dirout = basepath+'/anomaly'
# dirout = '/Volumes/Data_2TB/NMME/SST/by_lead/anomaly';

# Model names
#           1              2               3            4              5             6
mods = ['CanCM4i', 'COLA-RSMAS-CCSM4', 'GEM-NEMO', 'GFDL-SPEAR', 'NASA-GEOSS2S', 'NCEP-CFSv2']#
nmod = len(mods)
nl = dict(zip(mods,[11, 11, 11, 11, 8, 9])) # Max lead time for each model
mods = ['NCEP-CFSv2',]#, 'NCEP-CFSv2']#

In [3]:
# Loop through models
print('\nCalculating anomalies for NMME forecasts...\n')
for modi in mods:
    
    print(f'\nProcessing {modi}...\n')
    print( 'Lead')
    
    # Loop through lead times
    for il in range(0,nl[modi]):
        print(' ',il)
        
        # Load climatology
        f_clim = f'{dir_clim}/sst_{modi}_l{il}_ensmean_climatology_1991_2020.nc'
        fclim = xr.open_dataset(f_clim,decode_times=False)
        sst_clim = fclim.sst_clim.values
    
        # Load hindcast/forecast
        f_in = f'{dir_forc}/sst_{modi}_l{il}_concatenated.nc'
        fin = xr.open_dataset(f_in,decode_times=False)
        sst0=fin.sst.values
        time=fin.time.values
        lon=fin.lon.values
        lat=fin.lat.values
        
        # Constrain to specified years
        yy=np.array([int(el/12)+1960 for el in time])
        mm=np.array([int(el%12)+1 for el in time])
        ind = (yy>=clim_years[0]) & (yy<=clim_years[1]);
        sst = sst0[ind,:,:,:]
        yy = yy[ind]
        mm = mm[ind]
        time = time[ind];

        # Loop through time and compute anomalies
        nt = len(yy)
        
        sst_an=np.empty((nt,)+np.shape(sst)[1:])
        for it in range(0,nt):
            sst_an[it,:,:,:] = sst[it,:,:,:] - sst_clim[mm[it]-1,:,:]

        # Save to file
        f_out = f'{dirout}/sst_{modi}_l{il}_anomaly_{clim_years[0]}_{clim_years[1]}.nc'
        year = yy;
        month = mm;
        xout=xr.Dataset(data_vars={'lon':(['X',],lon),
                                'lat':(['Y',],lat),
                                'time':(['S'],time),
                                'year':(['S'],year),
                                'month':(['S'],month),
                                'sst_an':(['S','M','Y','X'],sst_an)},
                    coords=dict(X=fin.X,Y=fin.Y,M=fin.M,S=("S",time)),)
        xout.to_netcdf(f_out,mode='w')
        fin.close()
        fclim.close()
        del sst;del sst_an;del time;del year;del month
print('\nDone\n\n')


Calculating anomalies for NMME forecasts...


Processing NCEP-CFSv2...

Lead
  0
  1
  2
  3
  4
  5
  6
  7
  8

Done


